# <center> KIT Praktikum Neuronale Netze: Deep Neural Network Solution </center>

</br>
On this exercise, you are implementing a feedforward neural network from scratch to do the MNIST problem. This one is using <code>Numpy</code>. As an exercise for Backpropagation as the Computational Graph view you learned from the practical lecture of the workshop, you should work through the derivation of the loss function to the weights or biases with pen and paper.   
</br>

We will try do modularize our neural network as it consists of $L$ layers. **Read the following description carefully, otherwise you would have problems implementing it later**. Each layer $l$ has a weight matrix $W^{(l)}$ with the size of $(n^{(l)}, n^{(l-1)})$ where $n^{(l)}$ is the number of neurons (outputs) of the layer $l$, a bias vector $b^{(l)}$ with the size of $(n^{(l)}, 1)$. Each layer $l$ has an activation function $f$. Each layer $l$ can be the last layer or not (whose the error term is calculated differently). First, we will implement those parts for one layer:
    
* Initialize a layer: Use Xavier Initialization if the activation is sigmoid or tanh, otherwise use He Initialization.
* Do the forward pass through the layer based on its activation function:
    *  $$ z^{(l)} = W^{(l)}\cdot a^{(l-1)} + b^{(l)} $$
        $$ a^{(l)} = f(z^{(l)}) $$
    *  Where:<br/>
        $z^{(l)}$: net output of layer $l$<br/>
        $a^{(l)}$: activation of layer $l$, is the input of the next layer $l+1$, $a^{(0)} = X$<br/>
* Do the backward pass through the layer. Remember our convenient notion $\check{w} = \frac{\partial \mathcal{L}}{\partial w}$? Here in the code, $\check{w}$ is denoted by dw.<br/>
$$\check{w}^{(l)} = \check{a}^{(l)} \frac{\partial a^{(l)}}{\partial z^{(l)}}\frac{\partial z^{(l)}}{\partial w^{(l)}}= \check{a}^{(l)}f^{(l)'}a^{(l-1)}$$
* Update the weights/biases based on the gradient descent principle.

Then training the network is divided into several steps:
* Initialize the whole network (by initilizing every layer)
* Split training data into minibatches.
* Iteratively do following steps in $E$ epochs:
    * Shuffle minibatches.
    * Do the forward pass.
    * Calculate the loss.
    * Do the backward pass.
    * Update weights.
    
With the trained model, perform inference steps on the test data.  
   


First, let's import `numpy` and explore some basic methods: 

In [4]:
import numpy as np
import torch

# 2. Implement basic activation functions (vectorized version)

__Sigmoid__:

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

__Hyperbolic Tangent__:

In [6]:
def tanh(z):
    return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)

__Rectified Linear Unit (ReLU)__:

In [7]:
def relu(z):
    return np.maximum(0, z)

__Linear__:

In [8]:
def linear(z):
    return z

__Softmax__:

In [9]:
def softmax(z):
    ez = np.exp(z - np.max(z))
    return ez / np.sum(ez)
    # The original formular is less preferred due to numerical unstability:
    #ez = np.exp(z)
    #return ez / np.sum(ez)
    

__Softmax for batch__:
</br>

Note that if we do minibatch update, we must compute the softmax along some dimension that is not the batch dimension. From our convention, batch dimension is the second (axis=1), so we must compute the softmax along the first dimension (axis=0).

In [10]:
def softmax_batch(z, axis=0):
    ez = np.exp(z - np.max(z, axis=axis, keepdims=True))
    return ez / np.sum(ez, axis=axis, keepdims=True)
    # The original formular is less preferred due to numerical unstability:
    #ez = np.exp(z)
    #return ez / np.sum(ez, axis=axis, keepdims=True)
    

__Test your implementations__:

In [12]:
np.random.seed(1234)
z = np.random.randn(100,2)

sigmoid_z = sigmoid(z)
tanh_z = tanh(z)
my_tanh_z = np.tanh(z)
relu_z = relu(z)
linear_z = linear(z)
softmax_z = softmax(z)


print("Sigmoid results of z[0]=%f, z[6]=%f, z[8]=%f, z[22]=%f, z[98]=%f" \
      %(sigmoid_z[0,0], sigmoid_z[6,0], sigmoid_z[8,0], sigmoid_z[22,0], sigmoid_z[98,0]))
print("Tanh results of z[3]=%f, z[9]=%f, z[18]=%f, z[34]=%f, z[99]=%f" \
      %(tanh_z[3,0], tanh_z[9,0], tanh_z[18,0], tanh_z[34,0], tanh_z[99,0]))
print("Relu results of z[8]=%f, z[16]=%f, z[38]=%f, z[72]=%f, z[88]=%f" \
      %(relu_z[8,0], relu_z[16,0], relu_z[38,0], relu_z[72,0], relu_z[88,0]))
print("Linear results of z[2]=%f, z[6]=%f, z[8]=%f, z[22]=%f, z[98]=%f" \
      %(linear_z[2,0], linear_z[6,0], linear_z[8,0], linear_z[22,0], linear_z[98,0]))
print("Softmax results of z[2]=%f, z[6]=%f, z[8]=%f, z[22]=%f, z[98]=%f" \
      %(softmax_z[2,0], softmax_z[6,0], softmax_z[8,0], softmax_z[22,0], softmax_z[98,0]))

Sigmoid results of z[0]=0.615723, z[6]=0.721783, z[8]=0.599997, z[22]=0.561633, z[98]=0.459803
Tanh results of z[3]=0.696046, z[9]=0.867072, z[18]=0.124070, z[34]=0.208769, z[99]=0.280640
Relu results of z[8]=0.405453, z[16]=1.047579, z[38]=0.226963, z[72]=1.176812, z[88]=2.123692
Linear results of z[2]=-0.720589, z[6]=0.953324, z[8]=0.405453, z[22]=0.247792, z[98]=-0.161137
Softmax results of z[2]=0.001570, z[6]=0.008374, z[8]=0.004842, z[22]=0.004136, z[98]=0.002748


__Your results should be__: <br/>
Sigmoid results of z[0]=0.615723, z[6]=0.721783, z[8]=0.599997, z[22]=0.561633, z[98]=0.459803 <br/>
Tanh results of z[3]=0.696046, z[9]=0.867072, z[18]=0.124070, z[34]=0.208769, z[99]=0.280640 <br/>
Relu results of z[8]=0.405453, z[16]=1.047579, z[38]=0.226963, z[72]=1.176812, z[88]=2.123692 <br/>
Linear results of z[2]=-0.720589, z[6]=0.953324, z[8]=0.405453, z[22]=0.247792, z[98]=-0.161137 <br/>
Softmax results of z[2]=0.001570, z[6]=0.008374, z[8]=0.004842, z[22]=0.004136, z[98]=0.002748 <br/>

# 3. Implement the derivations of basic activation functions 


__Sigmoid__:

In [13]:
def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

__Tanh__:

In [14]:
def tanh_prime(z):
    return (1 - tanh(z)**2)

__Rectified Linear Unit (ReLU)__:

In [15]:
def relu_prime(z):
    return np.logical_not(z < 0)  # Theoretically if z = 0, relu_prime should return UNDEFINED

__Linear__:

In [16]:
def linear_prime(z):
    return 1

__Softmax__: 

This is hard to come up with without knowing the loss function. Most of the implementations out there are assuming we have cross entropy (or negative log likelihood) loss function, but it is not general enough, isn't it? So feel free to take my implementation. For your interest: https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/

In [17]:
def softmax_prime(z):
    y = softmax(z).reshape(-1,1)
    return np.diagflat(y) - np.dot(y, y.T)

__Softmax for batch__: 

In [40]:
def softmax_prime_batch(z, axis=0):

    result = np.empty((z.shape[axis], z.shape[axis],0))
    for i in range(z.shape[1-axis]):
        y1 = softmax_prime(z[:,i])
        result= np.concatenate((result,y1.reshape(z.shape[axis],z.shape[axis],1)), axis=(2-axis))
    
    return result

__Test your implementations__:

In [19]:
np.random.seed(8888)
z = np.random.randn(100,1)

sigmoid_prime_z = sigmoid_prime(z)
tanh_prime_z = tanh_prime(z)
relu_prime_z = relu_prime(z)
softmax_prime_z = softmax_prime(z)

# The derivative of a softmax should be a Jacobian matrix
print(softmax_prime_z.shape)

print("Sigmoid prime results of z[2]=%f, z[6]=%f, z[8]=%f, z[22]=%f, z[98]=%f" \
      %(sigmoid_prime_z[2,0], sigmoid_prime_z[6,0], sigmoid_prime_z[8,0], sigmoid_prime_z[22,0], sigmoid_prime_z[98,0]))
print("Tanh prime of z[3]=%f, z[9]=%f, z[18]=%f, z[34]=%f, z[99]=%f" \
      %(tanh_prime_z[3,0], tanh_prime_z[9,0], tanh_prime_z[18,0], tanh_prime_z[34,0], tanh_prime_z[99,0]))
print("Relu prime results of z[8]=%f, z[16]=%f, z[38]=%f, z[72]=%f, z[88]=%f" \
      %(relu_prime_z[8,0], relu_prime_z[16,0], relu_prime_z[38,0], relu_prime_z[72,0], relu_prime_z[88,0]))
print("Softmax prime results of z[8]=%f, z[16]=%f, z[38]=%f, z[72]=%f, z[88]=%f" \
      %(softmax_prime_z[8,0], softmax_prime_z[16,0], softmax_prime_z[38,0], softmax_prime_z[72,0], softmax_prime_z[88,0]))

(100, 100)
Sigmoid prime results of z[2]=0.247928, z[6]=0.228343, z[8]=0.181437, z[22]=0.249350, z[98]=0.133364
Tanh prime of z[3]=0.027258, z[9]=0.928484, z[18]=0.357361, z[34]=0.621576, z[99]=0.568175
Relu prime results of z[8]=1.000000, z[16]=0.000000, z[38]=1.000000, z[72]=0.000000, z[88]=1.000000
Softmax prime results of z[8]=-0.000087, z[16]=-0.000009, z[38]=-0.000030, z[72]=-0.000010, z[88]=-0.000031


__Your results should be__: <br/>
(100, 100)<br/>
Sigmoid prime results of z[2]=0.247928, z[6]=0.228343, z[8]=0.181437, z[22]=0.249350, z[98]=0.133364<br/>
Tanh prime of z[3]=0.027258, z[9]=0.928484, z[18]=0.357361, z[34]=0.621576, z[99]=0.568175<br/>
Relu prime results of z[8]=1.000000, z[16]=0.000000, z[38]=1.000000, z[72]=0.000000, z[88]=1.000000<br/>
Softmax prime results of z[8]=-0.000087, z[16]=-0.000009, z[38]=-0.000030, z[72]=-0.000010, z[88]=-0.000031

# 4. Implement basic cost (error) functions and their derivatives


Note that we are going to calculate the error for a (mini)batch of $m$ training examples, which we assume they are [independent and identically distributed](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables) (i.i.d), so the (normalized) total errors of those $m$ training examples (or $m$ data points) is $E = \frac{1}{m}\sum_{i=1}^mE^{(i)}$, where $E^{(i)}$ is an error value of the $i^{th}$ training example. However, in order to vectorize the error over a (mini)batch, we should come up with the direct version of error function of $E$ instead of $E^{(i)}$.

__Mean of Squared Error__:

In [20]:
def MeanSquaredError(t, y):
    # t: target label
    # y: predicted value
    n, m = y.shape
    return np.sum((y - t)**2) / (n * m)
    # or a better solution:
    #return np.mean(np.square(y - t))
    
def dMSE_dy(t, y):
    n, m = y.shape
    return 2 * (y-t) / (n * m)


__(Categorical) Cross Entropy Error__:

In [21]:
def CrossEntropyError(t, y , epsilon=1e-15):
    # t: target label of shape(n,m)
    # y: predicted value of shape(n,m)
    
    # To avoid numerical issues
    y = np.clip(y, epsilon, 1)
    
    # Remember to divide it to m
    m = y.shape[1]   
    return -np.sum(t * np.log(y)) / m

def dCE_dy(t, y):

    m = y.shape[1]
    return (y - t)/m

__Binary Cross Entropy Error__:

In [22]:
def BinaryCrossEntropyError(t, y , epsilon=1e-15):
    # t: target label  of shape(1,m)
    # y: predicted value of shape (1,m)
    
    # To avoid numerical issues
    y = np.clip(y, epsilon, 1)
    
    # Remember to divide it to m
    m = y.shape[1]   
    return np.sum(-t[0,:] * np.log(y[0,:]) - (1 - t[0,:]) * np.log(1 - y[0,:])) / m

def dBCE_dy(t, y):
    m = y.shape[1]
    return (y[0,:] - t[0,:]) / ((y[0,:] * m) * (1 - y[0,:])) 

__Test your implementations__:

In [23]:
# A handy helper function
def create_random_onehot(n, m):
    index = np.eye(n)
    return index[np.random.choice(index.shape[1], size=m)].T
t = create_random_onehot(3, 7)
print(t)

[[ 1.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  1.  1.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  1.]]


In [24]:
# Test forward

np.random.seed(1111)
y = np.random.randn(100,50)
sum_y = np.sum(y, axis=0, keepdims=True)
y = y / sum_y # Create softmax-like distribution
t = create_random_onehot(100,50)

# Test forward
myMSE = MeanSquaredError(t,y)
myCE = CrossEntropyError(t,y)

print("My Mean of Squared Error: " + str(myMSE))
print("My Cross Entropy Error: " + str(myCE))

yb = np.random.rand(100,50)
tb = np.random.randint(2, size=(100,50))

ybs = np.concatenate((yb, 1-yb))
tbs = np.concatenate((tb, 1-tb))


myBCE = BinaryCrossEntropyError(tb,yb)
print("My Binary Cross Entropy Error: " + str(myBCE))



My Mean of Squared Error: 0.42921158147
My Cross Entropy Error: 15.8303339039
My Binary Cross Entropy Error: 1.04915015195


__Your results should be__: <br/>
My Mean of Squared Error: 0.42921158147 <br/>
My Cross Entropy Error: 15.8303339039 <br/>
My Binary Cross Entropy Error: 1.04915015195

In [25]:
# Test backward

dMSE_dy1 = dMSE_dy(t, y)
dCE_dy1 =  dCE_dy(t, y)
dBCE_dy1 =  dBCE_dy(tb, yb)
    
print("My Mean of Squared Derivation dMSE_dy at dMSE_dy[2,4]=%f, dMSE_dy[6,5]=%f, dMSE_dy[8,20]=%f, dMSE_dy[22,7]=%f, dMSE_dy[98,40]=%f" \
      %(dMSE_dy1[2,4], dMSE_dy1[6,5], dMSE_dy1[8,20], dMSE_dy1[22,7], dMSE_dy1[98,40]))
print("My Cross Entropy Derivation: dCE_dy at dCE_dy[2,4]=%f, dCE_dy[6,5]=%f, dCE_dy[8,20]=%f, dCE_dy[22,7]=%f, dCE_dy[98,40]=%f" \
      %(dCE_dy1[2,4], dCE_dy1[6,5], dCE_dy1[8,20], dCE_dy1[22,7], dCE_dy1[98,40]))
print("My Binar Cross Entropy Derivation: dBCE_dy at dBCE_dy[2]=%f, dBCE_dy[6]=%f, dBCE_dy[20]=%f, dBCE_dy[22]=%f, dBCE_dy[40]=%f" \
      %(dBCE_dy1[2], dBCE_dy1[6], dBCE_dy1[20], dBCE_dy1[22], dBCE_dy1[40]))

My Mean of Squared Derivation dMSE_dy at dMSE_dy[2,4]=0.000000, dMSE_dy[6,5]=0.000618, dMSE_dy[8,20]=-0.000168, dMSE_dy[22,7]=0.000003, dMSE_dy[98,40]=0.000039
My Cross Entropy Derivation: dCE_dy at dCE_dy[2,4]=0.000018, dCE_dy[6,5]=0.030883, dCE_dy[8,20]=-0.008391, dCE_dy[22,7]=0.000130, dCE_dy[98,40]=0.001969
My Binar Cross Entropy Derivation: dBCE_dy at dBCE_dy[2]=0.100696, dBCE_dy[6]=-0.671816, dBCE_dy[20]=0.031237, dBCE_dy[22]=-0.106447, dBCE_dy[40]=-0.052379


__Your results should be__: <br/>
My Mean of Squared Derivation dMSE_dy at dMSE_dy[2,4]=0.000000, dMSE_dy[6,5]=0.000618, dMSE_dy[8,20]=-0.000168, dMSE_dy[22,7]=0.000003, dMSE_dy[98,40]=0.000039<br/>
My Cross Entropy Derivation: dCE_dy at dCE_dy[2,4]=0.000018, dCE_dy[6,5]=0.030883, dCE_dy[8,20]=-0.008391, dCE_dy[22,7]=0.000130, dCE_dy[98,40]=0.001969<br/>
My Binar Cross Entropy Derivation: dBCE_dy at dBCE_dy[2]=0.100696, dBCE_dy[6]=-0.671816, dBCE_dy[20]=0.031237, dBCE_dy[22]=-0.106447, dBCE_dy[40]=-0.052379<br/>

# 5. Layer Steps

## 5.1. Initialization

Initialize a layer: Use Xavier Initialization if the activation is sigmoid or tanh, otherwise use He Initialization.

In [26]:
def initialize(n_in, n_out, activation_string):
    """
    Implement the initialization for our layer

    Arguments:
    n_in -- size of previous layer
    n_out -- size of current layer
    activation_string -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    """
    
    if activation_string == 'sigmoid' or activation_string == 'tanh' or activation_string == 'softmax':
        # Xavier Initialization
        W = np.random.randn(n_out, n_in) * (np.sqrt(2. / (n_in + n_out)))
        b = np.zeros((n_out, 1))
    else: 
        # He Initialization
        W = np.random.randn(n_out, n_in) * (np.sqrt(1. / n_in))
        b = np.zeros((n_out, 1))
    return W, b

## 5.2. Forward pass

In [27]:
def forward(A_prev, W, b, activation_string):
    """
    Implement the forward propagation for our layer

    Arguments:
    A_prev -- activations from previous layer (or input data X): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation_string -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- values of (A_prev, Z, W) we store for computing backward propagation efficiently
    """
    
    Z = np.dot(W, A_prev) + b
    
    if activation_string == 'sigmoid':
        A = sigmoid(Z)
    elif activation_string == 'tanh':
        A = tanh(Z)
    elif activation_string == 'relu':
        A = relu(Z)
    elif activation_string == 'softmax':
        A = softmax_batch(Z, axis=1)
    else:
        A = linear(Z)
    
    cache = (A_prev, Z, W, activation_string)
    
    return A, cache
    

__Test forward__:

In [28]:
np.random.seed(2222)
A_prev = np.random.randn(3,2)
W = np.random.randn(1,3)
b = np.random.randn(1,1)

A, cache = forward(A_prev, W, b, "sigmoid")
print("With sigmoid: Z = %s , A = %s" % (str(cache[1]), str(A)))

A, _ = forward(A_prev, W, b, "tanh")
print("With tanh: A = " + str(A))

A, _ = forward(A_prev, W, b, "relu")
print("With ReLU: A = " + str(A))

A, _ = forward(A_prev, W, b, "softmax")
print("With softmax: A = " + str(A))

With sigmoid: Z = [[ 1.1337088  -1.20334105]] , A = [[ 0.75652269  0.2308814 ]]
With tanh: A = [[ 0.81228478 -0.83467086]]
With ReLU: A = [[ 1.1337088  0.       ]]
With softmax: A = [[ 0.91189936  0.08810064]]


__Your results should be__: <br/>
With sigmoid: Z = [[ 1.1337088  -1.20334105]] , A = [[ 0.75652269  0.2308814 ]]<br/>
With tanh: A = [[ 0.81228478 -0.83467086]]<br/>
With ReLU: A = [[ 1.1337088  0.       ]]<br/>
With softmax: A = [[ 0.91189936  0.08810064]]

## 5.3. Backward pass

In [29]:
def backward(dA, cache):
    """
    Implement the backward propagation for our layer.
    
    Arguments:
    dA -- activation gradient for current layer l 
    cache -- values of (A_prev, Z, W) we store for computing backward propagation efficiently
    activation_string -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    
    A_prev, Z, W, activation_string = cache
    m = A_prev.shape[1]
    
    if activation_string == 'sigmoid':
        dZ = dA * sigmoid_prime(Z)
    elif activation_string == 'tanh':
        dZ = dA * tanh_prime(Z)
    elif activation_string == 'relu':
        dZ = dA * relu_prime(Z)
    elif activation_string == 'softmax':  # This is hard, I did it for you
        dZ = dA * softmax_batch(Z, axis=1)
        s = dZ.sum(axis=dZ.ndim - 1, keepdims=True)
        dZ -= s * softmax_batch(Z, axis=1)
    else:
        dZ = dA * linear_prime(Z)
    
    dW = (1 / m) * np.dot(dZ, A_prev.T)
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db


__Test Backward__:

In [30]:
np.random.seed(1111)
dA = np.random.randn(1,2)
A_prev = np.random.randn(3,2)
W = np.random.randn(1,3)
b = np.random.randn(1,1)
Z = np.random.randn(1,2)

cache_sigmoid = (A_prev, Z, W, "sigmoid")
cache_tanh = (A_prev, Z, W, "tanh")
cache_relu = (A_prev, Z, W, "relu")
cache_softmax = (A_prev, Z, W, "softmax")

dA_prev, dW, db = backward(dA, cache_sigmoid)
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = backward(dA, cache_tanh)
print ("tanh:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = backward(dA, cache_relu)
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = backward(dA, cache_softmax)
print ("softmax:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.38601836  0.4093035 ]
 [ 0.04975095  0.05275199]
 [ 0.37804364  0.40084773]]
dW = [[ 0.01722623 -0.1803237   0.09144594]]
db = [[-0.25163402]]

tanh:
dA_prev = [[ 0.74460672  1.47719476]
 [ 0.09596665  0.19038431]
 [ 0.72922394  1.4466775 ]]
dW = [[ 0.22431658 -0.34262738  0.38665635]]
db = [[-0.70296173]]

relu:
dA_prev = [[ 2.05442539  0.        ]
 [ 0.26477914  0.        ]
 [ 2.01198317  0.        ]]
dW = [[-0.51363355 -0.97619193 -0.17936819]]
db = [[-0.65000516]]

softmax:
dA_prev = [[ 0.05452866 -0.05452866]
 [ 0.00702778 -0.00702778]
 [ 0.05340215 -0.05340215]]
dW = [[-0.02878513 -0.02632298 -0.02143347]]
db = [[ -1.38777878e-17]]


__Your results should be__: <br/>
sigmoid:<br/>
dA_prev = [[ 0.38601836  0.4093035 ]<br/>
 [ 0.04975095  0.05275199]<br/>
 [ 0.37804364  0.40084773]]<br/>
dW = [[-0.01627613 -0.14937127  0.091592  ]]<br/>
db = [[-0.25163402]]<br/>
<br/>
tanh:<br/>
dA_prev = [[ 0.74460672  1.47719476]<br/>
 [ 0.09596665  0.19038431]<br/>
 [ 0.72922394  1.4466775 ]]<br/>
dW = [[-0.25178866 -0.44375637  0.44336361]]<br/>
db = [[-0.70296173]]<br/>
<br/>
relu:<br/>
dA_prev = [[ 2.05442539  0.        ]<br/>
 [ 0.26477914  0.        ]<br/>
 [ 2.01198317  0.        ]]<br/>
dW = [[ 0.6115193  -0.30198246 -0.35733097]]<br/>
db = [[-0.65000516]]<br/>

softmax:<br/>
dA_prev = [[ 0.05452866 -0.05452866]<br/>
 [ 0.00702778 -0.00702778]<br/>
 [ 0.05340215 -0.05340215]]<br/>
dW = [[-0.02878513 -0.02632298 -0.02143347]]<br/>
db = [[ -1.38777878e-17]]<br/>

## 5.4. Update parameters:

In [31]:
def update_parameters(lr, W, b, dW, db):
    W1 = W - lr * dW 
    b1 = b - lr * db
    return W1, b1

# 6. Network Steps

## 6.1. Network Initialization

In [32]:
def network_initialize(layer_sizes = [768,100,50,10], activations = ["relu","relu","softmax"], seed=9999):
    """
    Initialize the parameters of a network.
    
    Arguments:
    layer_sizes -- A list of layer sizes. 
    activations -- A list of corresponding activation functions.
    
    Returns:
    parameters -- Dictionary of initialized weights and biases for every layers.
    """
    
    np.random.seed(seed)
    
    parameters = {} 
    
    for l in range(1, len(layer_sizes)):
        parameters['W' + str(l)], parameters['b' + str(l)] = initialize(layer_sizes[l-1], layer_sizes[l], activations[l-1])
        parameters['act' + str(l)] = activations[l-1]
        
    return parameters
        

In [33]:
import pprint
parameters = network_initialize([2,3,4],["relu","softmax"])
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(parameters)


{   'W1': array([[-0.36418784,  0.41853418],
       [ 0.03385161, -0.34535427],
       [-0.25098956, -0.27705387]]),
    'W2': array([[ 0.99648945, -0.73309211,  1.18999424],
       [-0.06313226,  0.06406165,  0.09760321],
       [-0.35157642, -0.87994782, -0.61020235],
       [-0.76922564,  0.39821514, -0.92965227]]),
    'act1': 'relu',
    'act2': 'softmax',
    'b1': array([[ 0.],
       [ 0.],
       [ 0.]]),
    'b2': array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])}


__Your results should be__: <br/>
{ &nbsp;  &nbsp;  &nbsp;  &nbsp;    'W1': array([[-0.36418784, &nbsp;  &nbsp;  0.41853418], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;       [ 0.03385161, &nbsp;  &nbsp; -0.34535427], <br/>
   &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;      [-0.25098956, &nbsp;  &nbsp; -0.27705387]]), <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp;    'W2': array([[ 0.99648945, &nbsp;  &nbsp; -0.73309211, &nbsp;  &nbsp;  1.18999424], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;       [-0.06313226,  &nbsp;  &nbsp; 0.06406165, &nbsp;  &nbsp;  0.09760321], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;      [-0.35157642, &nbsp;  &nbsp; -0.87994782, &nbsp;  &nbsp; -0.61020235], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;      [-0.76922564, &nbsp;  &nbsp;  0.39821514, &nbsp;  &nbsp; -0.92965227]]), <br/>
&nbsp;  &nbsp;  &nbsp;  &nbsp;    'act1': 'relu', <br/>
&nbsp;  &nbsp;  &nbsp;  &nbsp;     'act2': 'softmax', <br/>
&nbsp;  &nbsp;  &nbsp;  &nbsp;     'b1': array([[ 0.], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;      [ 0.], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;      [ 0.]]), <br/>
&nbsp;  &nbsp;  &nbsp;  &nbsp;   'b2': array([[ 0.], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;      [ 0.], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;      [ 0.], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;      [ 0.]])} <br/>

## 6.2. Network Forward 

In [34]:
def network_forward(X, parameters):
    """
    Do the forward pass through the layers.
    
    Arguments:
    X -- The input of the size (n, m): n features and m instances (m = batch size).
    parameters -- The weights and biases of every layers in the network.
    
    Returns:
    A -- Final activations (output activations).
    caches -- the cached values for faster calculation of the backward step later.
    """

    A_prev = X
    L = len(parameters) // 3 # Number of layers in the network
    caches = []
    
    for l in range(L):
        W = parameters['W' + str(l+1)]
        b = parameters['b' + str(l+1)]
        activation_string = parameters['act' + str(l+1)]
        A, cache = forward(A_prev, W, b, activation_string)
        caches.append(cache)
        A_prev = A
    
    return A, caches

In [35]:
np.random.seed(6)
X = np.random.randn(5,4)
W1 = np.random.randn(4,5)
b1 = np.random.randn(4,1)
W2 = np.random.randn(3,4)
b2 = np.random.randn(3,1)
W3 = np.random.randn(1,3)
b3 = np.random.randn(1,1)
  
parameters = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3,
              "act1": "relu",
              "act2": "relu",
              "act3": "softmax"}
AL, caches = network_forward(X, parameters)    
print("AL = " + str(AL))



AL = [[ 0.01497328  0.87662976  0.09019153  0.01820543]]


__Your results should be__: <br/>
AL = [[ 0.01497328  0.87662976  0.09019153  0.01820543]]

## 6.3. Calculate the error and error term of the last layer 

In [36]:
def calculate_error(T, AL, error_string):
    """
    Calculate the error and the error term of the last layer.
    
    Arguments:
    T -- The target labels of the size (n, m): n features and m instances (m = batch size).
    AL -- Final activations (output activations from the last layer).
    error_string -- The string representing the error function: "bce", "mse" or "ce".
    
    Returns:
    error -- The error value.
    dAL -- the error term (the derivative of the error w.r.t the activation) of the last layer.
    """
    if error_string == "mse":
        error = MeanSquaredError(T, AL)
        dAL = dMSE_dy(T, AL)
    elif error_string == 'bce':
        error = BinaryCrossEntropyError(T, AL)
        dAL = dBCE_dy(T, AL)
    elif error_string == 'ce':
        error = CrossEntropyError(T, AL)
        dAL = dCE_dy(T, AL)
    else:
        raise NameError("Your error string '%s' is undefined!" % error_string)
        
    return error, dAL
    

## 6.3. Network Backward

In [37]:
def network_backward(dAL, caches):
    """
    Do the backward pass through the layers.
    
    Arguments:
    dAL -- the error term (the derivative of the error w.r.t the activation) of the last layer.
    caches -- the cached values from the forward step before.
   
    Returns:
    grads -- a dictionary to store the calculated derivatives of the error w.r.t to the weights and biases.
    """
    L = len(caches) # Number of layers in the network
    grads = {}
    
    dA = dAL
    
    for l in reversed(range(1,L+1)):
        cache_l = caches[l-1]
        dA, grads["dW" + str(l)], grads["db" + str(l)] = backward(dA, cache_l)
        grads["dA" + str(l-1)] = dA
    
    return grads

__Test your implementations__:

In [38]:
np.random.seed(6)
X = np.random.randn(7,6)
W1 = np.random.randn(5,7)
b1 = np.random.randn(5,1)
W2 = np.random.randn(3,5)
b2 = np.random.randn(3,1)
W3 = np.random.randn(1,3)
b3 = np.random.randn(1,1)
  
parameters = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3,
              "act1": "relu",
              "act2": "sigmoid",
              "act3": "softmax"}

t = create_random_onehot(1,6)



AL, caches = network_forward(X, parameters)

print("My activation last layer AL:\n", AL)
error, dAL = calculate_error(t, AL, "ce")


print("========================")
print(" My error:" + str(error))

my_A2, my_Z3, _, _ = caches[2]

#my_dZ3 = dAL * sigmoid_prime(my_Z3)

my_dZ3 = dAL * softmax(my_Z3)
s = my_dZ3.sum(axis=my_dZ3.ndim - 1, keepdims=True)
my_dZ3 -= s * softmax(my_Z3) 



m = 1 # my_A2.shape[1]
my_dW3 = np.dot(my_dZ3, my_A2.T)
my_db3 = np.sum(my_dZ3, axis=1, keepdims=True)
my_dA2 = np.dot(W3.T, my_dZ3)


my_A1, my_Z2, _, _ = caches[1]
my_dZ2 = my_dA2 * relu_prime(my_Z2)
my_dW2 = np.dot(my_dZ2, my_A1.T)
my_db2 = np.sum(my_dZ2, axis=1, keepdims=True)
my_dA1 = np.dot(W2.T, my_dZ2)

my_A0, my_Z1, _, _ = caches[0]
my_dZ1 = my_dA1 * relu_prime(my_Z1)
my_dW1 = np.dot(my_dZ1, my_A0.T)
my_db1 = np.sum(my_dZ1, axis=1, keepdims=True)
my_dA0 = np.dot(W1.T, my_dZ1)


print("========================")
print("My W1 grad dW1:\n" + str(torch.from_numpy(my_dW1))) 
print("My b1 grad db1:\n" + str(torch.from_numpy(my_db1)))





My activation last layer AL:
 [[ 0.21921768  0.11988903  0.16260051  0.12001303  0.24447803  0.13380173]]
 My error:1.83258646479
My W1 grad dW1:
tensor([[-0.0010, -0.0042,  0.0002,  0.0012, -0.0027, -0.0008,  0.0011],
        [-0.0017,  0.0006,  0.0001,  0.0011,  0.0006,  0.0009,  0.0007],
        [-0.0009,  0.0015, -0.0012, -0.0013,  0.0005,  0.0006,  0.0004],
        [ 0.0012, -0.0007,  0.0009,  0.0004, -0.0005, -0.0013, -0.0013],
        [ 0.0002, -0.0002,  0.0003,  0.0005,  0.0001, -0.0000, -0.0003]],
       dtype=torch.float64)
My b1 grad db1:
tensor([[-0.0037],
        [ 0.0013],
        [-0.0020],
        [ 0.0017],
        [ 0.0007]], dtype=torch.float64)


__Your results should be__: <br/>
My activation last layer AL: <br/>
 &nbsp;  &nbsp; [[ 0.21921768  0.11988903  0.16260051  0.12001303  0.24447803  0.13380173]] <br/>
======================== <br/>
 &nbsp; My error:1.83258646479 <br/>
======================== <br/>
My W1 grad dW1: <br/>
tensor([[-0.0010, -0.0042,  0.0002,  0.0012, -0.0027, -0.0008,  0.0011], <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;         [-0.0017,  0.0006,  0.0001,  0.0011,  0.0006,  0.0009,  0.0007], <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;        [-0.0009,  0.0015, -0.0012, -0.0013,  0.0005,  0.0006,  0.0004], <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;          [ 0.0012, -0.0007,  0.0009,  0.0004, -0.0005, -0.0013, -0.0013], <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;         [ 0.0002, -0.0002,  0.0003,  0.0005,  0.0001, -0.0000, -0.0003]], <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp;        dtype=torch.float64) <br/>
My b1 grad db1: <br/>
tensor([[-0.0037], <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;         [ 0.0013], <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;         [-0.0020], <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;         [ 0.0017], <br/>
 &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;         [ 0.0007]], dtype=torch.float64) <br/>

# 6.4. Update the parameters:

In [39]:
def network_update(lr, parameters, grads):
    """
    Update the parameters of all layers.
    
    Arguments:
    lr -- learning rate.
    parameters -- the parameters of the network to be updated.
    grads -- a dictionary to store the calculated derivatives of the error w.r.t to the parameters.
   
    Returns:
    parameters -- the updated parameters of the network.
    """
    
    L = len(parameters) // 3 # Number of layers in the network
    for l in range(L):
        parameters["W" + str(l+1)] -= lr * grads["dW" + str(l+1)] 
        parameters["b" + str(l+1)] -= lr * grads["db" + str(l+1)]
        
    return parameters

__Test your implementations__:

In [ ]:
np.random.seed(6)
X = np.random.randn(7,10)
W1 = np.random.randn(5,7)
b1 = np.random.randn(5,1)
W2 = np.random.randn(3,5)
b2 = np.random.randn(3,1)
  
parameters = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "act1": "relu",
              "act2": "softmax"}

t = create_random_onehot(3,10)


AL, caches = network_forward(X, parameters)
print(np.sum(AL,axis=0))
error, dAL = calculate_error(t, AL, "ce")
grads = network_backward(dAL, caches)

parameters = network_update(0.5, parameters, grads)

import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(parameters)







__Your results should be__: <br/>
{&nbsp;  &nbsp;  &nbsp;    'W1': array([[-1.03295651,  0.35862528,  1.07353604, -0.37522946,  0.39616932, <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;       -0.47148865,  2.33632501], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;      [ 1.50286388, -0.59545956,  0.52839833,  0.93994217,  0.42634415, <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;        -0.75807972, -0.16243501], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;       [ 0.72680932,  0.44408241, -0.85682286,  0.44692782, -1.01464803, <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;      -2.13232319,  0.17386349], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [ 0.95139218,  0.4419052 ,  1.46915517,  1.7497915 ,  0.35367386, <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;      -0.64316151, -0.04739572], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [-1.44902964, -0.03619117, -0.09084896,  0.17629557,  1.09461738, <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp; &nbsp;  &nbsp;      -2.12647575,  0.75144374]]), <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;    'W2': array([[-0.73957208,  0.52294535, -0.59187648, -0.47748711,  0.11252968], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [ 1.90396151,  0.69458768, -0.01951732,  1.66320563,  0.02993579], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [-0.29749921, -0.96813764,  0.16706695,  0.11660199, -0.68225729]]), <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;    'act1': 'relu', <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;   'act2': 'softmax', <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;    'b1': array([[-0.54084024], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [ 0.79330547], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [ 0.17365274], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [-1.03523086], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [ 0.87426565]]), <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;    'b2': array([[-1.91402115], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [-0.13990231], <br/>
  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;  &nbsp;     [ 0.11492465]])} <br/>

# 7. Train MNIST

## 7.1. Read MNIST data

In [41]:
data_path = "../Data/mnist_seven.csv"
data = np.genfromtxt(data_path, delimiter=",", dtype="uint8")
train, dev, test = data[:4000], data[4000:4500], data[4500:]
print(train.shape)

(4000, 785)


## 7.2. Normalize data

In [44]:
def normalize(dataset):
    X = dataset[:, 1:] / 255.     # Normalize input features
    Y_temp = dataset[:, 0]
    
    # Convert labels to one-hot vectors
    n_values = np.max(Y_temp) + 1
    Y = np.eye(n_values)[Y_temp]

    return X.T, Y.T

In [45]:
X_train, Y_train = normalize(train)
print(X_train.shape)
print(Y_train.shape)

X_test, Y_test = normalize(test)
print(X_test.shape)
print(Y_test[:,5])

(784, 4000)
(10, 4000)
(784, 500)
[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]


## 7.3. Training

In [48]:
epochs = 100
batch_size = 200

# Train
print("Training...")
np.random.seed(1234)
parameters = network_initialize(layer_sizes = [784,100,50,10], activations = ["sigmoid", "sigmoid", "softmax"])


for epoch in range(epochs):
    L = 0
    
    # Split into minibatches into a *list* of sub-arrays
    # we want to split along the number of instances, so axis = 1
    X_minibatch = np.array_split(X_train, batch_size, axis = 1)
    Y_minibatch = np.array_split(Y_train, batch_size, axis = 1) 

    # We shuffle the minibatches of X and Y in the same way
    nmb = len(X_minibatch) # number of minibatches
    np.random.seed(8888)
    shuffled_index = np.random.permutation(range(nmb))

    # Now we can do the training, we cannot vectorize over different minibatches
    # They are like our "epochs"
    for i in range(nmb):
        X_current = X_minibatch[shuffled_index[i]]
        Y_current = Y_minibatch[shuffled_index[i]]         
            
    #   Those two commented lines are for training Batch GD   
    #   AL, caches = network_forward(X_train, parameters)
    #   error, dAL = calculate_error(Y_train, AL, "ce")
        AL, caches = network_forward(X_current, parameters)
        error, dAL = calculate_error(Y_current, AL, "ce")
        grads = network_backward(dAL, caches)
        parameters = network_update(15, parameters, grads)
        L += error
        
    print("Error of the epoch {0}: {1}".format(epoch + 1, L/batch_size))

print("...Done!")

Training...
Error of the epoch 1: 2.962905431085156
Error of the epoch 2: 2.8822520623100774
Error of the epoch 3: 2.707589912648429
Error of the epoch 4: 2.383859018345201
Error of the epoch 5: 2.0588371308092124
Error of the epoch 6: 1.8203479814149524
Error of the epoch 7: 1.6940501042316065
Error of the epoch 8: 1.6214515998747185
Error of the epoch 9: 1.5838952386206195
Error of the epoch 10: 1.564293865817791
Error of the epoch 11: 1.551468370175875
Error of the epoch 12: 1.5403775436200073
Error of the epoch 13: 1.530073374608231
Error of the epoch 14: 1.520545323572489
Error of the epoch 15: 1.5115586997830732
Error of the epoch 16: 1.5028758793058878
Error of the epoch 17: 1.4944096332868608
Error of the epoch 18: 1.486165634186042
Error of the epoch 19: 1.4781704261768858
Error of the epoch 20: 1.4704354874723946
Error of the epoch 21: 1.4629529234322995
Error of the epoch 22: 1.4557048780587873
Error of the epoch 23: 1.4486733017720832
Error of the epoch 24: 1.44184581379718

## 7.4. Testing

In [49]:
m_test = X_test.shape[1]
AL_test, _ = network_forward(X_test, parameters)
correct = (np.argmax(Y_test, axis=0) == np.argmax(AL_test, axis=0)).sum()
   
print('Accuracy on the test images: %d %%' % (100 * correct / m_test))

Accuracy on the test images: 93 %


It should produce a 93% accuracy, which is not bad. Just a little bit worse than the pytorch version (94%) whose difference is the usage of Adam (a better optimization technique) compared to our simple SGD. Notice the learning rate. Change it and play with it. In our simple SGD-based model, it is an important hyperparameter and the performance of our model is learning rate-sensitive. **Do you agree with me that PyTorch already helps us a lot on this simple feed-forward architecture?**